In [106]:
import geopandas as gpd
import rasterio
from shapely.geometry import Polygon

tif = rasterio.open("/home/kandelaki/git/SAM-Adapter-PyTorch/preprocessing/test_folder/images/NW00603.tif")
bbox = tif.bounds
mask = gpd.read_file("/home/kandelaki/git/SAM-Adapter-PyTorch/preprocessing/test_folder/geojson/PV_MUC_20220506_1_Florian.geojson", bbox = bbox)
mask = mask.to_crs(tif.crs)
mask.explore()

In [107]:
mask.to_file("/home/kandelaki/git/SAM-Adapter-PyTorch/preprocessing/test_folder/splits/" + "split_mask.geojson", driver='GeoJSON')

In [108]:
new_mask = gpd.read_file("/home/kandelaki/git/SAM-Adapter-PyTorch/preprocessing/test_folder/splits/" + "split_mask.geojson")
new_mask = gpd.GeoSeries(data=new_mask["geometry"], crs=mask.crs)

In [109]:
polygon_out_of_tile = Polygon.from_bounds(*tif.bounds)
polygon_out_of_tile = gpd.GeoSeries(data = polygon_out_of_tile, crs=mask.crs)
polygon_out_of_tile.set_crs(tif.crs, inplace=True)
polygon_out_of_tile.explore()

In [110]:
df1 = gpd.GeoDataFrame({'geometry': polygon_out_of_tile})
df2 = gpd.GeoDataFrame({'geometry': new_mask})
intersection = gpd.overlay(df1, df2, how='intersection')
intersection.explore()
patch_size = 1024


In [111]:
centroids = new_mask.centroid.to_crs(tif.crs)

In [112]:
centroids.explore()

In [128]:
distances = new_mask.geometry.apply(lambda x: new_mask.hausdorff_distance(x))

In [161]:
threshold_distance = 300  # adjust as needed
groups = []
grouped_indices = set()

for i, distance_row in distances.iterrows():
    if i not in grouped_indices:
        group = distance_row[distance_row <= threshold_distance].index.tolist()
        groups.append(group)
        grouped_indices.update(group)


In [162]:
groups

[[0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30],
 [31, 32, 33, 34, 35, 36],
 [37, 38, 39, 40, 41, 42, 43, 44, 45],
 [46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78],
 [79, 80, 81, 82],
 [83, 84, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102],
 [85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 103, 104, 105]]

In [163]:
from shapely.geometry import MultiPolygon
# unite points in each group into a single polygon
multipolygons = []
for group in groups:
    united = MultiPolygon()
    for i in group:
        print(new_mask.geometry[i])
        united = united.union(new_mask.geometry[i])
    multipolygons.append(united)

MULTIPOLYGON (((683953.7319436418 5348615.869156123, 684022.2059528952 5348624.022386263, 684022.7309093629 5348619.44542206, 683954.2240903303 5348610.882069681, 683953.7319436418 5348615.869156123)))
MULTIPOLYGON (((684021.8614502134 5348627.057290842, 684022.041903999 5348624.58015251, 683942.2813306903 5348615.16374587, 683942.2977355799 5348617.575264643, 684021.8614502134 5348627.057290842)))
MULTIPOLYGON (((684019.2202629849 5348650.056946081, 684019.6467901149 5348645.562006326, 683942.3797600282 5348636.145599687, 683942.215711132 5348640.804588337, 684019.2202629849 5348650.056946081)))
MULTIPOLYGON (((683942.3469502489 5348643.987136923, 684018.8183431904 5348653.223089776, 684018.8511529695 5348650.631117216, 683942.2813306905 5348641.329544805, 683942.3469502489 5348643.987136923)))
MULTIPOLYGON (((683953.2890116164 5348615.623082775, 683953.6335142984 5348610.849259896, 683942.1664964563 5348609.668107844, 683942.264925794 5348614.359906274, 683953.2890116164 5348615.6230

In [164]:
multipolygons

[<MULTIPOLYGON (((683953.732 5348615.869, 684022.206 5348624.022, 684022.731 ...>,
 <MULTIPOLYGON (((684028.62 5347701.116, 684028.44 5347699.082, 684029.818 53...>,
 <MULTIPOLYGON (((685783.369 5347347.624, 685773.986 5347349.248, 685767.604 ...>,
 <MULTIPOLYGON (((683985.935 5346817.614, 683965.658 5346814.629, 683965.527 ...>,
 <MULTIPOLYGON (((686321.909 5346723.417, 686327.29 5346721.892, 686325.945 5...>,
 <MULTIPOLYGON (((684439.71 5346489.131, 684441.581 5346481.634, 684438.037 5...>,
 <MULTIPOLYGON (((684033.968 5346388.668, 684048.339 5346390.259, 684048.405 ...>]

In [166]:
near_polygons = gpd.GeoSeries(multipolygons, crs=new_mask.crs)
near_polygons.explore()

In [176]:
enveloped = near_polygons.envelope
enveloped.explore()

In [172]:
polygon_out_of_tile.difference(enveloped[0]).explore()

In [175]:
enveloped

0    POLYGON ((683942.166 5348394.772, 684143.159 5...
1    POLYGON ((684026.192 5347672.211, 684178.446 5...
2    POLYGON ((685703.445 5347243.592, 685787.749 5...
3    POLYGON ((683965.527 5346748.107, 684007.753 5...
4    POLYGON ((686287.426 5346521.449, 686327.290 5...
5    POLYGON ((684200.945 5346360.656, 684466.590 5...
6    POLYGON ((684032.894 5346357.474, 684315.665 5...
dtype: geometry